In [ ]:
import os
import sys
# Get the directory where the current script is located
current_dir = os.path.dirname(os.getcwd()).split('\\')

# Construct the path to your target folder (e.g., 'data' inside the repo)
target_folder = "/".join(current_dir[:current_dir.index('src')+1])
sys.path.append(os.path.abspath(target_folder))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
height = 128
width = 128
size = height*width
windows_size = 5
path = "C:\VScode_Projects\DP\datasets\CIC-DDoS-2019\clean"
image_dir_name = "\image_bi_dir\\"
csv_file = path + "\cicddos2019_img_bi_dir.csv"

In [ ]:
df = pd.read_csv(path+"\labeled_sample.csv")

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
cols_to_move = ['src_ip', 'src_p', 'dst_ip', 'dst_p']
for col_move in cols_to_move:
    df = df[[col for col in df.columns if col != col_move] + [col_move]]

In [ ]:
df.protocol.unique()

In [ ]:
df.label.unique()

In [ ]:
len(df.label.unique())

In [ ]:
df.label.value_counts() / len(df)

In [ ]:
df = df.to_numpy()
df.shape

In [ ]:
# normalize payload, ttl, total_len
for col_index in range(df.shape[1]-7):
    column = df[:, col_index]
    column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column))
    df[:, col_index] = column_normalized

In [ ]:
# inspect t_delta
print(np.min(df[:, -6]), np.max(df[:, -6]))

In [ ]:
# normalize t_delta
column = df[:, -6]
column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column))
df[:, -6] = column_normalized

In [ ]:
def convert_dataset_to_grayscale_image(arr):
    for idx, row in enumerate(arr):
        print(row.shape)
        data = row[:-4]
        print(data.shape)
        idset = list(row[-4:]) + [row[-7]]
        srcip = idset[0]
        idset = frozenset(idset)
        print(idset, srcip)
        data = data[:-1]
        label = data[-1]
        print(data.shape)
        print(label)
        
        # data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        # print(data.shape)
        # data = data.reshape(height, width).astype('float64')
        # print(data.shape)
        
        # plt.imshow(data, cmap='gray')
        # plt.axis('off')  # Remove axes
        # plt.show()
        
        break
    print("DONE")
        
convert_dataset_to_grayscale_image(df)

In [ ]:
from sniffer.flow import FlowControl

In [ ]:
def gray_filter(img, tolerance):
    img = img
    R, G, B = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    gray_mask = np.abs(R - G) < tolerance
    img_no_gray = img.copy()
    img_no_gray[gray_mask] = [0, 0, 0]
    for c in range(3):
        channel = img_no_gray[:, :, c]
        mean = channel.mean()
        std = channel.std()
        if mean == 0 or std == 0:
            continue
        img_no_gray[:, :, c] = (channel - mean) / std
    return img_no_gray

In [ ]:
def convert_dataset_to_rgb_image(arr):
    flows = FlowControl()
    for idx, row in enumerate(arr):
        print(row.shape)
        idset = list(row[-4:]) + [row[-7]]
        srcip = idset[0]
        idset = frozenset(idset)
        print(idset, srcip)
        data = row[:-5]
        label = row[-5]
        print(data.shape)
        print(label)
        
        channel_1, channel_2 = flows.attach_dict(idset=idset, data=data, srcip=srcip, window_size=windows_size)
        
        channel_1 = np.array(channel_1).flatten()
        channel_1 = np.pad(channel_1, pad_width=int((size-len(channel_1))/2), constant_values=0)
        channel_1 = channel_1.reshape(height, width)
        
        channel_2 = np.array(channel_2).flatten()
        channel_2 = np.pad(channel_2, pad_width=int((size-len(channel_2))/2), constant_values=0)
        channel_2 = channel_2.reshape(height, width)
        
        channel_1 = channel_1.astype('float64')
        print(channel_1.shape)
        channel_2 = channel_2.astype('float64')
        print(channel_2.shape)
        channel_3 = channel_3 = np.zeros(shape=(height, width))
        print(channel_3.shape)
        img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
        print(img.shape)
        
        image = gray_filter(img, 0.1)
        
        plt.imshow(image)
        plt.axis('off')  # Remove axes
        plt.show() 
        break
    
    print("DONE")
        
convert_dataset_to_rgb_image(df)

In [ ]:
with open(csv_file, 'w') as f:
    np.savetxt(f, [np.array(["file_name", "label"])], delimiter=',', fmt='%s')

def convert_dataset_to_image(arr):
    flows = FlowControl()
    with open(csv_file, 'a') as f:
        for idx, row in enumerate(arr):
            idset = list(row[-4:]) + [row[-7]]
            srcip = idset[0]
            idset = frozenset(idset)
            data = row[:-5]
            label = row[-5]
        
            if idx % 10_000 == 0:
                    print(idx, label)
                    
            channel_1, channel_2 = flows.attach_dict(idset=idset, data=data, srcip=srcip, window_size=windows_size)
            channel_1 = np.array(channel_1).flatten()
            channel_2 = np.array(channel_2).flatten()
            
            channel_1 = np.pad(channel_1, pad_width=int((size-len(channel_1))/2), constant_values=0)
            channel_1 = channel_1.reshape(height, width)
            
            channel_2 = np.pad(channel_2, pad_width=int((size-len(channel_2))/2), constant_values=0)
            channel_2 = channel_2.reshape(height, width)
            
            channel_1 = channel_1.astype('float64')
            channel_2 = channel_2.astype('float64')
            channel_3 = np.zeros(shape=(height, width))
            img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
            
            file_name = f"cic_ddos_2019_{idx}.png"
            
            image = gray_filter(img, 0.1)
            cv2.imwrite(path+image_dir_name+file_name, image)
            
            log = np.array([file_name, label])
            np.savetxt(f, [log], delimiter=',', fmt='%s')
    print("DONE")
        
convert_dataset_to_image(df)